[실습문제] LLM 프롬프트로 Anki 단어장 생성하기 #32

`homework_anki_vocab_20250624.ipynb`에 작성하세요.


# LLM 프롬프트로 Anki 단어장 생성하기

### 실습 목표

* 영어 텍스트를 입력하면, 중요한 어휘를 추출해 Anki 카드 형식의 단어장 데이터를 생성하는 프롬프트를 설계한다.
* 프롬프트 내에 시스템 지시사항, 예시, 출력 형식을 포함한다.

### 문제

**다음 요구사항을 만족하는 LLM 프롬프트를 작성하시오.**

#### 요구사항

1. **시스템 지시사항**

   * LLM이 영어 텍스트에서 학습 가치가 있는 주요 단어 또는 어구를 추출하여 단어장으로 만들어야 한다.
   * 너무 쉬운 단어는 제외하고, 고유명사, 전문용어, 관용구 등도 포함한다.
   * 각 카드에는 (1)단어/어구, (2)뜻(영어 또는 한글), (3)품사, (4)예문(영어)을 포함한다.
   * 각 카드는 Anki에서 바로 쓸 수 있는 포맷(각 항목은 세미콜론 ; 으로 구분, 한 줄에 한 카드)으로 출력한다.

2. **예시**

   * 입력 텍스트 예시:

     ```
     The committee will convene next week to discuss the new policy proposals and reach a consensus.
     ```
   * 출력 예시:

     ```
     convene;to come together for a meeting;verb;The committee will convene next week.
     consensus;general agreement;noun;The group reached a consensus after a long discussion.
     proposal;a suggestion or plan;noun;He submitted a proposal for the new project.
     ```

3. **LLM 출력 형식**

   * 각 카드: `단어 또는 어구;뜻;품사;영어 예문`
   * 모든 카드는 한 줄에 한 개, 세미콜론(;)으로 필드 구분
   * 불필요한 부가설명, 줄바꿈, 문장 없음

4. **DataFrame 시각화**

    * LLM 응답을 파싱해서 다음과 같이 DataFrame으로 출력하세요.

![Image](https://private-user-images.githubusercontent.com/39724768/458282429-399dc257-7fc1-475a-9ffb-3cb408aa73b3.png?jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmF3LmdpdGh1YnVzZXJjb250ZW50LmNvbSIsImtleSI6ImtleTUiLCJleHAiOjE3NTA3NTUxMDUsIm5iZiI6MTc1MDc1NDgwNSwicGF0aCI6Ii8zOTcyNDc2OC80NTgyODI0MjktMzk5ZGMyNTctN2ZjMS00NzVhLTlmZmItM2NiNDA4YWE3M2IzLnBuZz9YLUFtei1BbGdvcml0aG09QVdTNC1ITUFDLVNIQTI1NiZYLUFtei1DcmVkZW50aWFsPUFLSUFWQ09EWUxTQTUzUFFLNFpBJTJGMjAyNTA2MjQlMkZ1cy1lYXN0LTElMkZzMyUyRmF3czRfcmVxdWVzdCZYLUFtei1EYXRlPTIwMjUwNjI0VDA4NDY0NVomWC1BbXotRXhwaXJlcz0zMDAmWC1BbXotU2lnbmF0dXJlPTJmZWIzZTY2MWM0ZDA5ODQ3OWFhNjFjNjZiYmM2MTAxNWE5ZjA5OTk3MzBlNmVhZmNkYmQ5N2EyYTZmNTM4Y2YmWC1BbXotU2lnbmVkSGVhZGVycz1ob3N0In0.-cYxXdCALtfSHVOs6lAXABN9mC_q3_2O_KakQnWanj4)



In [3]:
from google.colab import userdata
from openai import OpenAI

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)

In [4]:
system_message = """
너는 영어 어휘 학습용 단어장을 생성하는 어시스턴트야.
사용자가 입력한 영어 텍스트에서 핵심 어휘(중요하거나 자주 쓰이거나 고급 수준의 단어)를 추출하시오.
각 단어에 대해 다음 정보를 포함한 anki 카드 형식의 단어장 데이터를 생성하시오:

- 이 데이터는 고등학생 수준의 영어 어휘 학습용 anki 카드에 사용됩니다.
- 한글 해석은 자연스럽고 직관적으로, 예문은 실제 사용 가능한 문장으로 작성해주세요.
- 시험 출제 경향이 강한 단어위주로 추출합니다.

## Step ##
1. 고등학교 3학년 수준의 영어단어 목록을 100개 정도 검색합니다.
2. 검색한 단어들의 의미와 뉘앙스를 한글로 작성합니다.
3. 영어단어, 한글 뜻과 뉘앙스, 영어 예문, 품사를 세미콜론으로 구분된 형식으로 출력합니다.

## Output Format ##
검색결과를 아래 형식으로 출력합니다.

[영어단어];[한글 뜻];[품사];[발음기호];[영어예문]

## Example ##
cover;다루다,여행하다,포함하다,다루다,덮다;동사;[ˈkʌvə(r)];Snow covered the ground.

"""

user_message = """
Canada has signed a wide-ranging defence pact with the EU, as Donald Trump and global instability prompt traditional US allies to deepen their alliances.
Canadian prime minister, Mark Carney, on Monday joined European Commission president, Ursula von der Leyen, and head of the European Council, António Costa, in Brussels, where they signed a security and defence partnership, pledged more support for Ukraine, as well as joint work on issues from the climate crisis to artificial intelligence.
At a cordial press conference, Carney described Canada as “the most European of the non-European countries” that “looks first to the European Union to build a better world”.
Costa spoke in kind: “The European Union and Canada are among the closest allies in the transatlantic space. We see the world through the same lens. We stand for the same values.”
The French president, Emmanuel Macron, greets the Canadian PM, Mark Carney, in Paris in March.
Canada won’t become the 51st US state – but could it join the EU?
Read more
Not mentioned was another leader in the transatlantic space: Donald Trump, whose disrespect for old allies appears to have galvanised what was an already healthy EU-Canada relationship.
The US president is expected at the two-day Nato summit in the Hague starting on Tuesday, when members of the transatlantic alliance are called to pledge to spend 5% of GDP on defence.
Carney, a veteran central banker turned politician, won a stunning victory in April pledging that Canada would not become the 51st US state, a proposal often floated by Trump.
He said he had a mandate “to diversify and strengthen our international partnerships” and find new means of cooperation and coordination. The summit took place, Carney told reporters: “in a hinge moment of history, a world that is more dangerous and divided, a time where the rules-based international global order is under threat”.
The EU-Canada security and defence partnership opens the door to increased Canadian participation in the EU’s fledgling €150bn defence fund, known as Safe. Von der Leyen said the defence partnership meant working on joint capabilities, interoperability and joint procurement, referencing air defence. “The access of Canada to our joint procurement in the European Union, the door is open,” she said.
The security pact is a Canadian version of the agreement the EU signed with the British prime minister, Keir Starmer, last month. The bloc already had similar arrangements with six other countries including Norway and Japan, but this is the first with any country in the Americas. The defence pact includes joint work on cyber, maritime and space security, arms control and support for Ukraine.
Countries that have a defence and security pact with the EU can take part in joint procurement of weapons funded by the €150bn (£128bn, $173bn) Safe programme, although must negotiate a further technical agreement. Von der Leyen pledged both sides would “swiftly launch talks” on Canadian access to the joint procurement scheme.
Carney said the agreement with the EU would help Canada “deliver on our new capabilities more rapidly and more effectively”. Canada has been one of the laggards of the Nato alliance: in 2024 it spent just 1.37% of GDP on defence, well below the 2% set in 2014.
The two sides have a €125bn trading relationship, underpinned by the Ceta pact signed in 2016 that abolished 98% of tariffs. The agreement, however, has yet to be ratified by national parliaments in 10 EU member states, including Belgium, France, Italy and Poland, meaning elements of the deal have yet to enter into force.
In advance of the meeting, Carney and his wife, Diana Fox Carney, visited Schoonselhof military cemetery in Antwerp, where 348 Canadians are buried. On social media, Carney wrote: “brave young soldiers who ventured across the Atlantic to defend the freedom of Europe”.
The Carneys were accompanied by Belgium’s prime minister, formerly a long-serving Antwerp mayor, Bart De Wever, where they were given a tour of the ceremony and laid wreaths on behalf of Belgium and Canada. The last post was played by one of De Wever’s sons, according to local paper Het Nieuwsblad.
"""

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": system_message
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": user_message
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=0.5,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

print(response, "\n")
print(response.choices[0].message.content)

ChatCompletion(id='chatcmpl-Bltw7qmcKKl7ou8nSGLyNMLTmfQtB', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="defence;국방, 방어;명사;[dɪˈfɛns];The country has increased its defence spending in response to global threats.\n\npact;협정, 조약;명사;[pækt];The two nations signed a pact to promote trade and cooperation.\n\nalliance;동맹, 연합;명사;[əˈlaɪəns];The alliance between the countries strengthened their position in international negotiations.\n\ninteroperability;상호 운용성;명사;[ˌɪntəˌrɒpəˈbɪləti];The military exercises focused on improving the interoperability of NATO forces.\n\ncooperation;협력, 협조;명사;[koʊˌɒpəˈreɪʃən];International cooperation is essential for addressing climate change.\n\ngalvanise;자극하다, 고무하다;동사;[ˈɡælvənaɪz];The crisis galvanised the community into action to support those in need.\n\ncapabilities;능력, 역량;명사;[ˌkeɪpəˈbɪlɪtiz];The new technology enhances the military's capabilities in various operations.\n\nprocurement;조달, 구매;명사;[prəˈkjʊəmənt

In [ ]:
import re
import pandas as pd

result = response.choices[0].message.content
result = re.sub(r"\n\n", "\n", result)
anki_list = result.split("\n")
display(pd.DataFrame([anki.split(";") for anki in anki_list]), columns=["단어", "의미", "품사", "발음기호", "예문"])